# Fuzzing Certificates

In this notebook, I will try to decipher and fuzz digital certificates, also known as x.509 certificates.

In [ ]:
import bookutils

We'll start by _parsing_ certificates.

## Human-Readable Certificates

This is my certificate, coming in PEM format.

In [ ]:
CERT_PEM = '''
-----BEGIN CERTIFICATE-----
MIIGGDCCBQCgAwIBAgIMJGWlX/uTZaODs3uIMA0GCSqGSIb3DQEBCwUAMIGNMQsw
CQYDVQQGEwJERTFFMEMGA1UECgw8VmVyZWluIHp1ciBGb2VyZGVydW5nIGVpbmVz
IERldXRzY2hlbiBGb3JzY2h1bmdzbmV0emVzIGUuIFYuMRAwDgYDVQQLDAdERk4t
UEtJMSUwIwYDVQQDDBxERk4tVmVyZWluIEdsb2JhbCBJc3N1aW5nIENBMB4XDTIx
MDMwODEzMzQwOVoXDTI0MDMwNzEzMzQwOVowgbkxCzAJBgNVBAYTAkRFMREwDwYD
VQQIDAhTYWFybGFuZDEVMBMGA1UEBwwMU2FhcmJydWVja2VuMUQwQgYDVQQKDDtD
SVNQQSAtIEhlbG1ob2x0ei1aZW50cnVtIGZ1ZXIgSW5mb3JtYXRpb25zc2ljaGVy
aGVpdCBnR21iSDEQMA4GA1UEBAwHQW5kcmVhczEPMA0GA1UEKgwGWmVsbGVyMRcw
FQYDVQQDDA5aZWxsZXIgQW5kcmVhczCCASIwDQYJKoZIhvcNAQEBBQADggEPADCC
AQoCggEBALo6h1hJw92V8MnN38ry/Spc2G6v+YTTWsWODY2/7pPBizYIefCjDXCL
mkxu7oDUwm7Mbeg+gASeI1wJYpiKc8FknPkMDEAHYmZFpLbyfWJsCRTsu1WEO5So
2nOvedPTjpy7IwHHG7p7H9l5LCzWcA0XBqaGNj0yHwpOD67CA8jcbZ5I41dG3xUW
ApM51M+UqOAzhh0SlRkEgBnRE06jUj+zTVIKlLb9Ho9Bw3CEPdKRpBs6yjouKxwf
apwpaBru/NnEW7gwAwKNIoRlZmiKQniHoE4uMM7e5zWPxaV/co+cn3u8SwnqJjG0
jR07EF+l9Fb3cWGwSQrhp0lFjudp0aUCAwEAAaOCAkgwggJEMD4GA1UdIAQ3MDUw
DwYNKwYBBAGBrSGCLAEBBDAQBg4rBgEEAYGtIYIsAQEECDAQBg4rBgEEAYGtIYIs
AgEECDAJBgNVHRMEAjAAMA4GA1UdDwEB/wQEAwIF4DAdBgNVHSUEFjAUBggrBgEF
BQcDAgYIKwYBBQUHAwQwHQYDVR0OBBYEFJlDyZ5yzYQjHgDxf31rCqDITfLkMB8G
A1UdIwQYMBaAFGs6mIv58lOJ2uCtsjIeCR/oqjt0MBoGA1UdEQQTMBGBD3plbGxl
ckBjaXNwYS5kZTCBjQYDVR0fBIGFMIGCMD+gPaA7hjlodHRwOi8vY2RwMS5wY2Eu
ZGZuLmRlL2Rmbi1jYS1nbG9iYWwtZzIvcHViL2NybC9jYWNybC5jcmwwP6A9oDuG
OWh0dHA6Ly9jZHAyLnBjYS5kZm4uZGUvZGZuLWNhLWdsb2JhbC1nMi9wdWIvY3Js
L2NhY3JsLmNybDCB2wYIKwYBBQUHAQEEgc4wgcswMwYIKwYBBQUHMAGGJ2h0dHA6
Ly9vY3NwLnBjYS5kZm4uZGUvT0NTUC1TZXJ2ZXIvT0NTUDBJBggrBgEFBQcwAoY9
aHR0cDovL2NkcDEucGNhLmRmbi5kZS9kZm4tY2EtZ2xvYmFsLWcyL3B1Yi9jYWNl
cnQvY2FjZXJ0LmNydDBJBggrBgEFBQcwAoY9aHR0cDovL2NkcDIucGNhLmRmbi5k
ZS9kZm4tY2EtZ2xvYmFsLWcyL3B1Yi9jYWNlcnQvY2FjZXJ0LmNydDANBgkqhkiG
9w0BAQsFAAOCAQEAS7Ok9N8qAVgG6t5fa6rMEY4xU2DYIh1Xx8rXgHUa25ULktde
z+hEL2/3GRpA9fiQBccjJ3YVTuE0HuZ0hixbZie4L2aetQMrAO2wTzak42PGww5l
ERbtacNuW7t64s/LmLROKsWeeLDYChyJW1Ql5Wl7kkI9NV1BRPcGgtHqqhQ3CN/J
V4wK0JWPpD1lIQo/IaN/4RXq6unMZ/u1ZbXosXc8NlphAee1W2ZHI4ObWbpvpdBR
sj6PGMKESyLzODcuRMjib+qryiTp1e3PGmunmqS+kjNDsd3iohGQlej/Dsxx9gHT
UbbHFCEoTsnxEte5FcC1djFLrpQxklinOh/xnA==
-----END CERTIFICATE-----
'''

We can view the contents of this certificate using `openssl`:

In [ ]:
CERT_PEM_FILE = 'certificate.pem'

In [ ]:
with open(CERT_PEM_FILE, 'w') as fp:
    fp.write(CERT_PEM)

In [ ]:
!openssl x509 -in {CERT_PEM_FILE} -text -noout 

## Analyzing Binary Certificates

Let us convert this certificate into DER (binary) form. This is what we will work with.

In [ ]:
CERT_DER_FILE = 'certificate.der'

In [ ]:
!openssl x509 -outform der -in {CERT_PEM_FILE} -out {CERT_DER_FILE}

We can analyze file contents again using `openssl`:

In [ ]:
!openssl asn1parse -i -in {CERT_DER_FILE} -inform DER

To decode ASN.1, we make use of the [Python ASN.1 module](https://github.com/andrivet/python-asn1) (Here's the [documentation](https://python-asn1.readthedocs.io/en/latest/)).
We find that our DER file consists of a sequence (0x10 = 16): 

In [ ]:
import asn1

In [ ]:
CERT_DER = open(CERT_DER_FILE, 'br').read()
CERT_DER[:10]

In [ ]:
decoder_1 = asn1.Decoder()
decoder_1.start(CERT_DER)

In [ ]:
tag_1, value_1 = decoder_1.read()

In [ ]:
tag_1

In [ ]:
assert tag_1.nr == asn1.Numbers.Sequence
assert tag_1.typ == asn1.Types.Constructed

In [ ]:
value_1[:10]

This starts with another sequence:

In [ ]:
decoder_2 = asn1.Decoder()
decoder_2.start(value_1)

In [ ]:
tag_2, value_2 = decoder_2.read()

In [ ]:
tag_2

In [ ]:
assert tag_2.nr == asn1.Numbers.Sequence
assert tag_1.typ == asn1.Types.Constructed

In [ ]:
value_2[:10]

This starts with another sequence:

In [ ]:
decoder_3 = asn1.Decoder()
decoder_3.start(value_2)

In [ ]:
tag_3, value_3 = decoder_3.read()

In [ ]:
tag_3

In [ ]:
assert tag_3.typ == asn1.Types.Constructed
assert tag_3.cls == asn1.Classes.Context

In [ ]:
value_3

This is an encoding for an integer 2:

In [ ]:
decoder_4 = asn1.Decoder()
decoder_4.start(value_3)

In [ ]:
tag_4, value_4 = decoder_4.read()

In [ ]:
tag_4

In [ ]:
value_4

In [ ]:
assert tag_4.nr == asn1.Numbers.Integer
assert tag_4.typ == asn1.Types.Primitive

## A Class for ASN1 Parsing and Producing

We introduce a class `ASN1Parser` which we will use for parsing.

See [A Layman's Guide to a Subset of ASN.1, BER, and DER](http://luca.ntop.org/Teaching/Appunti/asn1.html) for details on the ASN1 format.

In [ ]:
from Grammars import Grammar, crange, is_valid_grammar, convert_ebnf_grammar
from GrammarFuzzer import display_tree

In [ ]:
from isla.solver import ISLaSolver, DerivationTree

In [ ]:
class ASN1Parser:
    def __init__(self, log=False):
        self.log = log
        pass

For each tag, we define length and value expansions.

In [ ]:
from pprint import pprint

In [ ]:
class ASN1Parser(ASN1Parser):
    TAG_ID_TO_STRING_MAP = {
        asn1.Numbers.Boolean: "BOOLEAN",
        asn1.Numbers.Integer: "INTEGER",
        asn1.Numbers.BitString: "BIT STRING",
        asn1.Numbers.OctetString: "OCTET STRING",
        asn1.Numbers.Null: "NULL",
        asn1.Numbers.ObjectIdentifier: "OBJECT",
        asn1.Numbers.PrintableString: "PRINTABLESTRING",
        asn1.Numbers.IA5String: "IA5STRING",
        asn1.Numbers.UTF8String: "UTF8STRING",
        asn1.Numbers.UnicodeString: "UNICODESTRING",
        asn1.Numbers.UTCTime: "UTCTIME",
        asn1.Numbers.GeneralizedTime: "GENERALIZED TIME",
        asn1.Numbers.Enumerated: "ENUMERATED",
        asn1.Numbers.Sequence: "SEQUENCE",
        asn1.Numbers.Set: "SET"
    }

    def tag_to_name(self, tag):
        if tag in self.TAG_ID_TO_STRING_MAP:
            name = self.TAG_ID_TO_STRING_MAP[tag].lower()
        else:
            name = "other"
        return name.replace(' ', '-')

    def tag_to_symbol(self, tag):
        return f'<{self.tag_to_name(tag)}>'

## Parsing ASN1

For parsing certificates, we'll use the Python `asn1` parser, and convert its result into a derivation tree. This is way more efficient than having the FuzzingBook `EarleyParser` first determine all alternatives and then have ISLa sort out those alternatives whose lengths match the constraints.

In [ ]:
class ASN1Parser(ASN1Parser):
    def parse(self, inp, skip_check=True):
        trees = self.decode(inp)
        return trees

    def new_tree(self, tag, value, children=None):
        name = self.tag_to_name(tag.nr)
        length = len(value)

        if not children:
            children = [DerivationTree(value)]

        tag_byte = tag.nr | tag.typ | tag.cls
        tag_tree = DerivationTree(f'<{name}-tag>',
                                  [DerivationTree(chr(tag_byte))])
        length_tree = DerivationTree(f'<{name}-length>',
                                    [self.new_length_tree(length)])
        value_tree = DerivationTree(f'<{name}-value>', children)

        tree = DerivationTree(f'<{name}>',
                              [tag_tree, length_tree, value_tree])
        return tree

    def decode(self, inp):
        trees = []
        decoder = asn1.Decoder()
        if isinstance(inp, str):
            inp = bytes(inp, 'latin-1')
        decoder.start(inp)

        while not decoder.eof():
            tag = decoder._read_tag()
            length = decoder._read_length()
            value = str(decoder._read_bytes(length), 'latin1')
            assert len(value) == length

            if tag.typ == asn1.Types.Primitive:
                # FIXME: This must be properly set for all primitive types
                tree = self.new_tree(tag, value)
            elif tag.typ == asn1.Types.Constructed:
                children = self.decode(value)
                tree = self.new_tree(tag, value, children)

            trees.append(tree)

        return trees

Encoding lengths in ASN1 is somewhat complex.

In [ ]:
class ASN1Parser(ASN1Parser):
    def new_length_tree(self, length):
        if length < 128:
            return DerivationTree('<length>',
                       [DerivationTree('<short-length>',
                            [DerivationTree('<byte0-127>',
                                 [DerivationTree(chr(length))])])])
        values = []
        while length:
            values.append(length & 0xff)
            length >>= 8
        values.reverse()
        assert len(values) < 127

        children = [DerivationTree('<byte128-255>',
                        [DerivationTree(chr(0x80 | len(values)))])]

        for val in values:
            children += [DerivationTree('<byte>',
                             [DerivationTree(chr(val))])]

        return DerivationTree('<length>',
                   [DerivationTree('<long-length>', children)])

In [ ]:
parser = ASN1Parser(log=False)

In [ ]:
tree = parser.parse(CERT_DER)[0]

In [ ]:
display_tree(tree)

In [ ]:
assert str(tree) == str(CERT_DER, 'latin1')

## Producing ASN1

To _produce_ ASN1, we introduce a class `ASN1Solver` that builds on the parser introduced above as well as the `ISLaSolver` class.

First, we need to construct the grammar, refining `create_grammar()`.

In [ ]:
class ASN1Solver(ASN1Parser, ISLaSolver):
    def __init__(self, formula='true', *args, log=False, start_symbol=None, start_tag=None, **kwargs):
        ASN1Parser.__init__(self, log)

        if not start_symbol and start_tag:
            start_symbol = self.tag_to_symbol(start_tag)

        self._grammar: Grammar = {
            '<start>': ['<value>'],
            '<value>': [],  # will be updated later
            }

        self._initial_formula = formula
        self._constraints = {}
        self._used_tags = set()
        self._start_tag = start_tag
        self._start_symbol = start_symbol

        self.create_grammar()

        grammar = self.trim_grammar(start_symbol)
        grammar = self.bnf_grammar()
        formula = self.make_formula()

        if self.log > 1:
            print("Grammar:", grammar)
            print("Formula:", formula)

        ISLaSolver.__init__(self, grammar, formula, *args, start_symbol=start_symbol, **kwargs)

    def trim_grammar(self, start_symbol=None):
        grammar = self.grammar.copy()
        constraints = self.constraints.copy()

        if start_symbol and start_symbol != '<start>':
            grammar['<start>'] = [start_symbol]
        for elem in unreachable_nonterminals(grammar):
            del grammar[elem]
            if elem in constraints:
                del constraints[elem]

        self._grammar = grammar
        self._constraints = constraints

    def bnf_grammar(self):
        bnf_grammar = convert_ebnf_grammar(self.grammar)
        return bnf_grammar

    @property
    def grammar(self):
        return self._grammar

    @grammar.setter
    def grammar(self, value):
        self._grammar = value

    @property
    def used_tags(self):
        return self._used_tags

    @property
    def constraints(self):
        return self._constraints

    def make_formula(self):
        formula = self._initial_formula
        for c in self.constraints.values():
            if c:
                if formula:
                    formula += '\nand\n'
                formula += c

        return formula

    def create_grammar(self):
        ...

We start with simple values.

### Simple Types

Every tag identifier can have bits 4-7 set, identifying types and classes. For efficient parsing, we enumerate the variations explicitly in the grammar (rather than identifying via constraints).

In [ ]:
class ASN1Solver(ASN1Solver):
    def tag_variations(self, tag, types=None):
        variations = []
        for typ_tag, typ in [
                ('<constructed>', asn1.Types.Constructed),   # 0x20
                ('<primitive>', asn1.Types.Primitive)        # 0x00
        ]:
            if types is not None and typ not in types:
                continue

            for cls_tag, cls in [
                ('<universal>', asn1.Classes.Universal),     # 0x00
                ('<application>', asn1.Classes.Application), # 0x40
                ('<context>', asn1.Classes.Context),         # 0x80
                ('<private>', asn1.Classes.Private)          # 0xc0
            ]:
                variations += [chr(tag | typ | cls) +
                               typ_tag + cls_tag]

        return variations

    def create_grammar(self):
        super().create_grammar()
        self.grammar.update({
          '<constructed>': [''],
          '<primitive>': [''],

          '<universal>': [''],
          '<application>': [''],
          '<context>': [''],
          '<private>': [''],
        })

In [ ]:
class ASN1Solver(ASN1Solver):
    def add_tag(self, tag, name=None, expansions=None, length=None, types=None,
                parent_symbol='<value>', check_grammar=True):
        if name is None:
            name = self.tag_to_name(tag)

        if expansions is None:
            expansions = ['<any-value>']
            self.grammar.update({
                '<any-value>': ['<byte>*'],
            })

        if length is None:
            length = ['<length>']
            self.add_length()

        self.add_defaults()

        assert f'<{name}>' not in self.grammar
        if parent_symbol:
            assert f'<{name}>' not in self.grammar[parent_symbol]

        new_rules = {
            f'<{name}>': [
                f'<{name}-tag><{name}-length><{name}-value>'
            ],
            f'<{name}-tag>': self.tag_variations(tag, types),
            f'<{name}-length>': length,
            f'<{name}-value>': expansions,
        }
        new_constraints = self.length_constraint(name)

        if parent_symbol:
            self.grammar[parent_symbol].append(f'<{name}>')

        self.grammar.update(new_rules)
        self.constraints[f'<{name}>'] = new_constraints
        self.used_tags.add(chr(tag))

        if self.log:
            print(f"New tag: <{name}> ({tag})")
            print("New rules:")
            pprint(new_rules)
            print("\nNew constraints:")
            print(new_constraints)

        assert f'<{name}>' in self.grammar
        if parent_symbol:
            assert f'<{name}>' in self.grammar[parent_symbol]

        if check_grammar:
            grammar = self.grammar.copy()
            for elem in unreachable_nonterminals(grammar):
                del grammar[elem]

            assert is_valid_grammar(grammar)

    # This is a tad too long
    # def length_constraint(self, name):
    #     return f'''forall <{name}>:
    # str.to_code(<{name}>.<{name}-length>) =
    #     str.len(<{name}>.<{name}-value>)'''

    # This should work, but does not (FIXME)
    def length_constraint(self, name):
        return f'str.to_code(<{name}>.<{name}-length>) = str.len(<{name}>.<{name}-value>)'

    def add_defaults(self):
        self.grammar.update({
            '<byte>': crange('\x00', '\xff'),
        })

In [ ]:
class ASN1Solver(ASN1Solver):
    EMIT_LONG_LENGTHS = False

    def add_length(self):
        self.grammar.update({
            '<byte0-127>': crange('\x00', '\x7f'),
        })

        if self.EMIT_LONG_LENGTHS:
            self.grammar.update({
                '<length>': ['<short-length>', '<long-length>'],
                '<short-length>': ['<byte0-127>'],
                '<long-length>': ['<byte128-255><byte>+'],
                '<byte128-255>': crange('\x80', '\xff'),
            })
        else:
            self.grammar.update({
                '<length>': ['<short-length>'],
                '<short-length>': ['<byte0-127>']
            })

ISLa has trouble fulfilling all the length constraints. Here's a workaround:

In [ ]:
class ASN1Solver(ASN1Solver):
    FIX_LENGTHS = True

    def length_constraint(self, name):
        if self.FIX_LENGTHS:
            return ''  # Let fix_lengths() handle this
        return super().length_constraint(name)

    def fix_lengths(self, tree):
        if not self.FIX_LENGTHS:
            return tree

        if not tree.children:
            return tree

        # Apply recursively
        new_children = []
        for c in tree.children:
            new_children.append(self.fix_lengths(c))

        if (len(new_children) == 3 and
            new_children[0].value.endswith('-tag>') and
            new_children[1].value.endswith('-length>') and
            new_children[2].value.endswith('-value>')):
                correct_length = len(str(new_children[2]))
                new_length_tree = self.new_length_tree(correct_length)
                new_children[1] = DerivationTree(new_children[1].value,
                                                 [new_length_tree])

        tree = DerivationTree(tree.value, new_children)

        return tree

    def solve(self, *args, **kwargs):
        tree = super().solve(*args, **kwargs)
        tree = self.fix_lengths(tree)
        return tree

### Tests

In [ ]:
from Grammars import unreachable_nonterminals

In [ ]:
from ExpectError import ExpectError

In [ ]:
class ASN1Solver(ASN1Solver):
    def test(self, *args, **kwargs):
        print(f"Test solving:")
        tree = self.solve(*args, **kwargs)
        print(f"Solution: {repr(str(tree))} {len(str(tree))}")

        parsed_tag = None
        parsed_value = None

        with ExpectError():
            print(f"\nTest decoding:")
            decoder = asn1.Decoder()
            decoder.start(bytes(str(tree), 'latin1'))
            parsed_tag, parsed_value = decoder.read()
            print("Decoding successful")

        if parsed_tag is not None and parsed_value is not None:
            print(f"Found {parsed_tag}, {parsed_value}", end="")
            if self._start_tag and parsed_tag.nr == self._start_tag:
                print("(expected)")
            if self._start_tag:
                assert parsed_tag.nr == self._start_tag

        return tree

#### Booleans

Booleans are simply `0x00` (False) or `0xff` (True)

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_boolean()

    def add_boolean(self):
        self.add_tag(
            tag=asn1.Numbers.Boolean,
            length=['\x01'],
            expansions=['\x00', '\xff'],
        )

In [ ]:
solver = ASN1Solver(start_tag=asn1.Numbers.Boolean, log=2)

In [ ]:
tree = solver.test()
# display_tree(tree)

#### Constructed Types

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_sequence()

    def add_sequence(self):
        self.add_tag(
            tag=asn1.Numbers.Sequence,
            expansions=['<value>+'],
            types=[asn1.Types.Constructed],
        )

In [ ]:
solver = ASN1Solver(log=True, start_tag=asn1.Numbers.Sequence)
tree = solver.test()
# display_tree(tree)

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_set()

    def add_set(self):
        self.add_tag(
            tag=asn1.Numbers.Set,
            expansions=['<value>+'],
            types=[asn1.Types.Constructed],
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.Set)
tree = solver.test()
# display_tree(tree)

#### Integers

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_integer()

    def add_integer(self):
        self.add_tag(
            tag=asn1.Numbers.Integer,
            length=['\x01', '\x02'],
            expansions=['<byte>+'],
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.Integer)
tree = solver.test()
# display_tree(tree)

FIXME: Add constraints re: integers

#### Null

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_null()

    def add_null(self):
        self.add_tag(
            name='null',
            tag=asn1.Numbers.Null,
            length=['\x00'],
            expansions=['']
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.Null)
tree = solver.test()
# display_tree(tree)

#### Unstructured Types

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_bit_string()

    def add_bit_string(self):
        self.add_tag(
            tag=asn1.Numbers.BitString,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.BitString)
tree = solver.test()
# display_tree(tree)

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_octet_string()

    def add_octet_string(self):
        self.add_tag(
            tag=asn1.Numbers.OctetString,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.OctetString)
tree = solver.test()
# display_tree(tree)

#### Object Identifiers

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_object_identifier()

    def add_object_identifier(self):
        self.add_tag(
            tag=asn1.Numbers.ObjectIdentifier,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.ObjectIdentifier)
tree = solver.test()
# display_tree(tree)

#### Enumerations

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_enumerated()

    def add_enumerated(self):
        self.add_tag(
            tag=asn1.Numbers.Enumerated,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.Enumerated)
tree = solver.test()
# display_tree(tree)

#### Strings

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_utf8_string()

    def add_utf8_string(self):
        self.add_tag(
            tag=asn1.Numbers.UTF8String,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.UTF8String)
tree = solver.test()
# display_tree(tree)

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_printable_string()

    def add_printable_string(self):
        self.add_tag(
            tag=asn1.Numbers.PrintableString,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.PrintableString)
tree = solver.test()
# display_tree(tree)

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_ia5_string()

    def add_ia5_string(self):
        self.add_tag(
            tag=asn1.Numbers.IA5String,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.IA5String)
tree = solver.test()
# display_tree(tree)

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_unicode_string()

    def add_unicode_string(self):
        self.add_tag(
            tag=asn1.Numbers.UnicodeString,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.UnicodeString)
tree = solver.test()
# display_tree(tree)

#### Time

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_utc_time()

    def add_utc_time(self):
        self._grammar.update({
            '<time>': ['<year><month-day><hour><minute><second>Z'],
            '<year>': ['<digit><digit>', '<digit><digit><digit><digit>'],
            '<month-day>': ['<month31><day1-31>',
                            '<month30><day1-30>',
                            '<month29><day1-29>'],
            '<month31>': ['01', '03', '05', '07', '08', '10', '12'],
            '<month30>': ['04', '06', '09', '11'],
            '<month29>': ['02'],
            '<hour>': ['<digit0-1><digit>' '20', '21', '22', '23'],
            '<minute>': ['<digit0-5><digit>'],
            '<second>': ['<digit0-5><digit>'],  # no leap seconds (60)
            '<digit0-1>': ['0', '1'],
            '<digit0-5>': ['0', '1', '2', '3', '4', '5'],
            '<digit>': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
            '<day1-31>': list(map("{:02}".format, range(1, 32))),
            '<day1-30>': list(map("{:02}".format, range(1, 31))),
            '<day1-29>': list(map("{:02}".format, range(1, 30))),
        })

        self.add_tag(
            tag=asn1.Numbers.UTCTime,
            expansions=['<time>']
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.UTCTime)
tree = solver.test()
# display_tree(tree)

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_generalized_time()

    def add_generalized_time(self):
        self.add_tag(
            tag=asn1.Numbers.GeneralizedTime,
        )

In [ ]:
solver = ASN1Solver(log=False, start_tag=asn1.Numbers.GeneralizedTime)
tree = solver.test()
# display_tree(tree)

### Other Types

We define a generic means to read in values whose tags we haven't seen before

In [ ]:
class ASN1Solver(ASN1Solver):
    def create_grammar(self):
        super().create_grammar()
        return self.add_other()

    def unused_tags(self):
        unused_tags = set()
        for tag in range(0, 0x1f):
            if chr(tag) not in self._used_tags:
                unused_tags.add(chr(tag))
        return unused_tags

    def add_other(self):
        self._grammar.update({
            '<other>': ['<other-tag><other-length><other-value>'],
            '<other-tag>': 
                ['<other-low-tag>', 
                 '<other-high-tag>'],
            '<other-low-tag>': list(self.unused_tags()),
            '<other-high-tag>': ['<high-tag><byte>+'],
            '<high-tag>': self.tag_variations(0x1f),

            '<other-length>': ['<length>'],
            '<other-value>': ['<byte>*'],
            })
        self._grammar['<value>'].append('<other>')

In [ ]:
solver = ASN1Solver(log=False, start_symbol='<other>')
tree = solver.test()
# display_tree(tree)

## Producing

In [ ]:
solver = ASN1Solver(log=False)

In [ ]:
solver.grammar

In [ ]:
tree = solver.solve()

In [ ]:
display_tree(tree)

## Producing X.509 Certificates

[Official X.509 standard](https://www.itu.int/rec/dologin_pub.asp?lang=e&id=T-REC-X.509-200811-S!!PDF-E&type=items)

[Intro to X509 fields](https://learn.microsoft.com/en-us/azure/iot-hub/tutorial-x509-certificates)

FIXME: include and support object identifiers [reference](https://learn.microsoft.com/en-us/windows/win32/seccertenroll/about-object-identifier?redirectedfrom=MSDN)

In [ ]:
class X509Solver(ASN1Solver):
    def create_grammar(self):
        # First, create the grammar for ASN1 fields
        super().create_grammar()

        # Now adjust for X509
        self.grammar.update({
            '<start>': ['<certificate>'],

            '<certificate>': ['<sequence-tag>'
                              '<sequence-length>'
                              '<certificate-value>'],
        })

        self.grammar.update({
            '<certificate-value>': ['<version>'
                                    '<serialNumber>'
                                    '<signature>'
                                    '<issuer>'
                                    '<validity>'
                                    '<subject>'
                                    '<subjectPublicKeyInfo>'
                                    '<issuerUniqueIdentifier>?'  # in v2 or v3
                                    '<subjectUniqueIdentifier>?'  # in v2 or v3
                                    '<extension>*'  # in v3
                                   ],

            '<version>': ['<integer>'],  # 0 is v1, 1 is v2, 2 is v3
            '<serialNumber>': ['<integer>'],
            '<signature>': ['<value>'],
            '<issuer>': ['<value>'],

            '<validity>': ['<sequence-tag><sequence-length><validity-value>'],
            '<validity-value>': ['<notBefore>'
                                 '<notAfter>'],
            '<notBefore>': ['<time>'],
            '<notAfter>': ['<time>'],
            '<time>': ['<utctime>', '<generalized-time>'],

            '<subject>': ['<string>'],
            '<string>': ['<utf8string>', '<printablestring>'],

            '<subjectPublicKeyInfo>': ['<value>'],

            '<issuerUniqueIdentifier>': ['<value>'],
            '<subjectUniqueIdentifier>': ['<value>'],
            '<extension>': ['<value>'],
        })

In [ ]:
solver = X509Solver(log=False)

In [ ]:
tree = solver.solve()

In [ ]:
display_tree(tree)